# RevenueBase-THT

## Process HTML

#### Load WARC files

In [6]:
def open_response(url_pickle_filepath): #(url_pickle_filepath=f"responses_{domain_url}.pickle"):
    with open(url_pickle_filepath, "rb") as f:
        all_pages = pickle.load(f)
    return all_pages

In [7]:
def warc_to_html_convertor(warc):
    with io.BytesIO(warc.content) as stream:
        for record in warcio.ArchiveIterator(stream):
            html = record.content_stream().read()#.decode('utf-8')#.decode('unicode_escape') #("utf-8")
    return html

In [8]:
def load_html(url_pickle_filepath):
    all_pages = open_response(url_pickle_filepath)
    for page in all_pages:
        html = warc_to_html_convertor(page)
        
    return html

In [9]:
def print_response_as_html(html):
    soup = BeautifulSoup(html, 'html')
    print(soup.prettify())

#### Analyze WARC files

In [10]:
def extract_metadata(html, warc_pickle_name):
    metadata = extruct.extract(
                        html,
                        base_url=get_base_url(html, warc_pickle_name),
                        syntaxes=['json-ld'],
                    )['json-ld'][0]
    return metadata

In [11]:
# Company Name QUERY (add this as a table record FINAL TABLE)

def extract_name(metadata):
    
    try:
        name = metadata['@graph'][0]['legalName']
    except:
        if len(metadata['publisher']['name'])>0:
            name = metadata['publisher']['name']
        elif len(metadata['legalName'])>0:
            name = metadata['legalName']
        else:
            name = dict({name: 'no_legal_name_found'})
    return name

In [12]:
# add this as a table record FINAL TABLE

def extract_address(metadata):
    
    try:
        if len(metadata['@graph'][0]['address'])>0:
            adrs = metadata['@graph'][0]['address']
        elif len(metadata['contactPoint'][0])>0:
            adrs = metadata['contactPoint'][0]
    except:
        adrs = 'no_contact_found'
    
    return adrs
    

In [13]:
def find_links_on_webpage(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile("^https://")}):
        links.append(link.get('href')) 
        
    return links

In [14]:
def is_app_available(links):
    matching = [True for link in links if "play.google" in link.lower() or 'apps.apple' in link.lower()]
    any_matching = any(x == True for x in matching)
    return any_matching

In [15]:
def is_demo_available(links):
    matching = [True for link in links if "demo" in link.lower()] #this could be improved as current "demo" regex lookup will return True if demo is a partial word as for ex. democracy 
    any_matching = any(x == True for x in matching)
    return any_matching

In [16]:
def process_warc(domain_list):
    
    results = []
    
    for domain in domain_list:
        warc_pickle_name = f"responses_{domain}.pickle"
        html = load_html(warc_pickle_name)
        
        # get metadata from html
        metadata = extract_metadata(html, warc_pickle_name)
        # extract name
        name = extract_name(metadata)
        # extract address
        address = extract_address(metadata)
        
        # find links on domain page
        links = find_links_on_webpage(html)
        # is there a demo
        is_demo = is_demo_available(links)
        # is there an app
        is_app = is_app_available(links)
        
        results.append([domain, name, address, is_demo, is_app ])
        
    return results

In [17]:
resu = process_warc(domain_list[:2])
resu

[['blogspot.com', 'Blogger', 'no_contact_found', False, False],
 ['wordpress.com',
  'Automattic, Inc.',
  {'@type': 'PostalAddress',
   'streetAddress': '60 29th Street #343',
   'addressLocality': 'San Francisco',
   'addressRegion': 'CA',
   'postalCode': '94110',
   'addressCountry': 'US'},
  False,
  True]]

In [1]:
results_df = pd.DataFrame(resu, columns = ['domain', 'company_name', 'contact', 'is_demo_available', 'is_app_available']) 


NameError: name 'pd' is not defined